# Seguimiento de objetos

Este cuaderno ensaya el seguimiento de objetos en video (asumiendo que la posición de la cámara está fija).

Salidas:
- Archivos CSV.
- Video 

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../videoanalytics/src")
import videoanalytics
videoanalytics.__version__

'0.0.2'

In [2]:
from datetime import datetime

## Chequeo de disponibilidad de HW / GPU

In [3]:
!nvidia-smi

Sat Aug 14 17:58:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Quadro M...  On   | 00000000:02:00.0 Off |                  N/A |
| 46%   31C    P8    12W / 120W |     62MiB /  8126MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA Quadro R...  On   | 00000000:81:00.0 Off |                  Off |
| 34%   

Detectadas por tensorflow en conda (FIXME).

In [4]:
import tensorflow as tf
print("GPUs disponibles: ", len(tf.config.list_physical_devices('GPU')))

GPUs disponibles:  0


## Datos de entrada y modelos

In [10]:
WORKSPACE_PATH="../"
WORKSPACE_DATA_PATH=WORKSPACE_PATH+"/data/"
INPUT_VIDEO_PATH=WORKSPACE_DATA_PATH+"media/videos-youtube/"
!ls {INPUT_VIDEO_PATH}

'buque EL MARISCO II ,proceso de captura merluz.mp4'
'LANGOSTINOS  PUERTO RAWSON  ENERO 201.mp4'
'Maniobra de pesca de merluza en mares argentino.mp4'
 maniobr.mp4
'Pesca artesanal de anchoita en mar del plat.mp4'
'pesca de altura en el mar argentino,pesquero.mp4'
'pesca de langostino,buque Alve.mp4'
'Pesca de langostino en aguas nacionale.mp4'
'pesca de langostino en rawson chubut con barco combenciona.mp4'
'PESCA DE LANGOSTINO RW CHUBU.mp4'
'PESCA EN RAWSO.mp4'
'Un dia de pesca merluza  Buque Paola.mp4'
'WhatsApp Video 2021-07-25 at 10.21.14 PM.mp4'
'WhatsApp Video 2021-07-25 at 9.24.42 PM.mp4'


Repositorio de modelos (local).

In [11]:
MODELS_PATH=WORKSPACE_DATA_PATH+"models/"
!ls {MODELS_PATH}

feature_extractor_ll.tar.gz  kaggle-fisheries-yolo  README.md
feature_extractor_mars	     mirtar-yolo


In [7]:
!tree {MODELS_PATH}/kaggle-fisheries-yolo

..//data/models//kaggle-fisheries-yolo
├── checkpoints
│   └── yolo-416
│       ├── assets
│       ├── saved_model.pb
│       └── variables
│           ├── variables.data-00000-of-00001
│           └── variables.index
├── kaggle-fisheries-perf-report.txt
├── kaggle-fisheries-yolo4.cfg
├── kaggle-fisheries-yolo4.weights
├── obj.names
└── README.md

4 directories, 8 files


In [8]:
!tree {MODELS_PATH}/mirtar-yolo

..//data/models//mirtar-yolo
├── checkpoints
│   └── yolo-416
│       ├── assets
│       ├── saved_model.pb
│       └── variables
│           ├── variables.data-00000-of-00001
│           └── variables.index
├── mirtar-yolo4.cfg
├── mirtar-yolo4.weights
├── obj.names
└── README.md

4 directories, 7 files


In [12]:
!tree {MODELS_PATH}/feature_extractor_mars

..//data/models//feature_extractor_mars
└── mars-small128.pb

0 directories, 1 file


## Definición de cadenas de procesamiento

### Seguimiento de objetos

In [36]:
DEEPSORT_MODEL_FILENAME = MODELS_PATH+"/feature_extractor_mars/mars-small128.pb"

In [29]:
from videoanalytics.pipeline import Pipeline
from videoanalytics.pipeline.sources import VideoReader
from videoanalytics.pipeline.sinks import VideoWriter

from videoanalytics.pipeline.sinks.object_detection import DetectionsAnnotator, DetectionsCSVWriter
from videoanalytics.pipeline.sinks.object_detection.yolo4 import YOLOv4DetectorTF

from videoanalytics.pipeline.sinks.object_tracking import TrackedObjectsAnnotator, TrackedObjectsCSVWriter
from videoanalytics.pipeline.sinks.object_tracking.sort import SORT
from videoanalytics.pipeline.sinks.object_tracking.deepsort import DeepSORT

def make_tracking_pipeline(input_video_filename,
                                output_det_csv_filename,
                                output_track_csv_filename,
                                output_video_filename,
                                detector_model_weights_filename,
                                detector_model_classes_filename,
                                tracker_type="DEEPSORT",
                                start_frame=0,
                                max_frames=None):
    context = {}
    pipeline = Pipeline()

    pipeline.add_component( VideoReader( "input",context,
                     video_path=input_video_filename,
                     start_frame=start_frame,
                     max_frames=max_frames))    
    
    pipeline.add_component( YOLOv4DetectorTF("detector",
                                             context,
                                             weights_filename=detector_model_weights_filename) )
        
    #pipeline.add_component( DetectionsCSVWriter("det_csv_writer",context,filename=output_det_csv_filename) )

    def make_sort_tracker():
        return SORT("tracker", context)
    
    def make_deep_sort_tracker():
        return DeepSORT("tracker", context, model_filename=DEEPSORT_MODEL_FILENAME)
    
    tracker_factory = {
        "SORT": make_sort_tracker(),
        "DEEPSORT": make_deep_sort_tracker()
    }
    
    pipeline.add_component( tracker_factory[tracker_type] )        
    
    pipeline.add_component( TrackedObjectsAnnotator("annotator",context) )
    pipeline.add_component( TrackedObjectsCSVWriter("tracker_csv_writer",context, 
                                                    filename = output_track_csv_filename) )
    
    pipeline.add_component(VideoWriter("writer",context,filename=output_video_filename))
    
    pipeline.set_connections([
        ("input", "detector"),
        ("detector", "tracker"),
        ("tracker", "tracker_csv_writer"),
        ("tracker", "annotator"),
        ("annotator", "writer")
    ])    
    
    return context,pipeline

## Batch job

Directorio de resultados.

In [25]:
!ls {WORKSPACE_DATA_PATH}/output

kaggle-fisheries-yolo  mirtar-yolo


In [43]:
INPUT_VIDEOS = [
#    (
#        'buque EL MARISCO II ,proceso de captura merluz.mp4',
#        None,
#    ),
#    
#    (
#        'LANGOSTINOS  PUERTO RAWSON  ENERO 201.mp4',
#        None,        
#    ),
#    
#    (
#        'Maniobra de pesca de merluza en mares argentino.mp4',
#        None,
#    ),
    
    ( 
        'maniobr.mp4',
        None
    ),
    
 #   (
 #       'Pesca artesanal de anchoita en mar del plat.mp4',
 #       None
 #   ),
 #   
 #   (
 #       'pesca de altura en el mar argentino,pesquero.mp4',
 #       None
 #   ),
 #   
 #   (
 #       'pesca de langostino,buque Alve.mp4',
 #       None
 #   ),
 #   
 #   (
 #       'Pesca de langostino en aguas nacionale.mp4',
 #       None
 #   ),
 #   
    (
        'pesca de langostino en rawson chubut con barco combenciona.mp4',
        None,
    ),
 #   
 #   (
 #       'PESCA DE LANGOSTINO RW CHUBU.mp4',
 #       None,
 #   ),
 #   
 #   (
 #       'PESCA EN RAWSO.mp4',
 #       None
 #   ),
 #   
 #   (
 #       'Un dia de pesca merluza  Buque Paola.mp4',
 #       None
 #   ),
 #   
 #   (
 #       'WhatsApp Video 2021-07-25 at 10.21.14 PM.mp4',
 #       None
 #   ),
 #   
 #   (
 #       'WhatsApp Video 2021-07-25 at 9.24.42 PM.mp4',
 #       None
 #   )
]

In [46]:
from os import path

def object_tracking_batch_job(input_videos,output_path,tracker_type="SORT",dry_run=False):
    obj_det_model = "mirtar-yolo"
    t0 = datetime.now()
    print("Inicio de proceso: ", t0)
    
    for i,v in enumerate(input_videos):
        input_video_filename=v[0]        
        output_video_filename=output_path+"job{:d}.mp4".format(i)
        output_det_csv_filename=output_path+"job{:d}-detections.csv".format(i)
        output_track_csv_filename=output_path+"job{:d}-trackings.csv".format(i)        

        print("Job {}. Procesando {}.".format(i, input_video_filename))
        
        if path.exists(output_video_filename):
            print("El archivo a generar {} ya existe. Pasando al siguiente".format(output_video_filename))    
            continue

        # FIXME, por ahora son fijos
        params=v[1]

        detector_model_weights_filename=MODELS_PATH+"/{}/checkpoints/yolo-416/".format(obj_det_model)
        detector_model_classes_filename=MODELS_PATH+"/{}/obj.names".format(obj_det_model)

        if not dry_run:
            
            context,pipeline=make_tracking_pipeline(
                input_video_filename=INPUT_VIDEO_PATH+input_video_filename,
                output_det_csv_filename=output_det_csv_filename,
                output_track_csv_filename = output_track_csv_filename,                
                output_video_filename=output_video_filename,
                detector_model_weights_filename=detector_model_weights_filename,
                detector_model_classes_filename=detector_model_classes_filename,
                tracker_type=tracker_type,
                start_frame=0,
                max_frames=None
            )

            pipeline.execute()
            print("Tiempo total de ejecución [s]:", pipeline.get_total_execution_time())
            print("Video de salida guardado en {}".format(output_video_filename))
    
    t1 = datetime.now()
    print("Fin del proceso:",t1)        
    print("Tiempo transcurrido:",t1-t0)

### Job con dataset Mirta-R y tracker SORT 

Nota: se aplica a los dos videos con cámara fija.

In [51]:
input_videos = INPUT_VIDEOS
output_base_path=WORKSPACE_DATA_PATH+"/output/"
obj_det_model = "mirtar-yolo"
output_path = output_base_path+obj_det_model+"-tracking-sort/"

In [48]:
# Reset
#!rm -rf {output_path}
#!mkdir {output_path}

In [52]:
object_tracking_batch_job(input_videos,output_path,tracker_type="SORT",dry_run=False)

Inicio de proceso:  2021-08-14 18:35:03.583829
Job 0. Procesando maniobr.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 6043.4097453839495
Video de salida guardado en ..//data//output/mirtar-yolo-tracking-sort/job0.mp4
Job 1. Procesando pesca de langostino en rawson chubut con barco combenciona.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 1563.7897116469685
Video de salida guardado en ..//data//output/mirtar-yolo-tracking-sort/job1.mp4
Fin del proceso: 2021-08-14 20:42:26.588154
Tiempo transcurrido: 2:07:23.004325


### Job con dataset Mirta-R y tracker DeepSORT

Nota: se aplica a los dos videos con cámara fija.

In [53]:
input_videos = INPUT_VIDEOS
output_base_path=WORKSPACE_DATA_PATH+"/output/"
obj_det_model = "mirtar-yolo"
output_path = output_base_path+obj_det_model+"-tracking-deepsort/"

In [41]:
# Reset
#!rm -rf {output_path}
#!mkdir {output_path}

In [54]:
object_tracking_batch_job(input_videos,output_path,tracker_type="DEEPSORT",dry_run=False)

Inicio de proceso:  2021-08-14 20:42:26.669939
Job 0. Procesando maniobr.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 8859.433763373003
Video de salida guardado en ..//data//output/mirtar-yolo-tracking-deepsort/job0.mp4
Job 1. Procesando pesca de langostino en rawson chubut con barco combenciona.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 1889.9920556879952
Video de salida guardado en ..//data//output/mirtar-yolo-tracking-deepsort/job1.mp4
Fin del proceso: 2021-08-14 23:42:13.216142
Tiempo transcurrido: 2:59:46.546203
